In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/train.csv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/test.csv /content

In [0]:
import pandas as pd

prefix = '/content/'
test_df = pd.read_csv(prefix + 'test.csv', header=None)
test_df=test_df.drop(index=0)

test_df = pd.DataFrame({
    'id':test_df[0],
    'text': test_df[1]
})

# display 
test_df.head()

In [0]:
!pip install allennlp
!pip install nltk
!pip install tqdm

from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/elmo-constituency-parser-2018.03.14.tar.gz")

START AGAIN

In [0]:
def extract_modal(obj):
    arr = list() # full set of constituents
    modal= list() # only the modal phrases
    past = list() # only the past phrases
    noun = list()
    verb = list()

    def extract(obj, arr):
        
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr)
                  
                if k=='nodeType' and v=='NP':
                    arr.append(v)
                if k=='nodeType' and v=='MD':
                    arr.append(v)
                if k=='nodeType' and v=='VP':    
                    arr.append(v)
                if k=='nodeType' and v=='VBD': # Verb, past tense
                    arr.append(v) 
                if k=='nodeType' and v=='ADVP': # Adverb Phrase
                    arr.append(v)           
                if k=='nodeType' and v=='RB':   # Adverb 
                    arr.append(v)
                if k=='nodeType' and v=='IN':    
                    arr.append(v)        

        elif isinstance(obj, list):
            for item in obj:

              if isinstance(item, dict):
                  if 'MD' in arr: 
                    arr.remove('MD')
                    modal.append(obj)
                  if 'NP' in arr: 
                    arr.remove('NP')
                    noun.append(obj)  
                  if 'VP' in arr:
                    arr.remove('VP')
                    verb.append(obj)
                    if 'wish' in obj[0]['word'].lower():
                      past.append(obj)  
                  if 'VBD' in arr:
                    arr.remove('VBD')
                    if 'were' in obj[0]['word'].lower():
                      past.append(obj)
                    if 'was' in obj[0]['word'].lower():
                      past.append(obj)  
                    if 'had' in obj[0]['word'].lower():
                      past.append(obj)
                  if 'ADVP' in arr:
                    arr.remove('ADVP')  
                    modal.append(obj)  
                  if 'RB' in arr: 
                    arr.remove('RB')
                    if 'even' in obj[0]['word'].lower():
                      past.append(obj)
                    if 'whichever' in obj[0]['word'].lower():
                      past.append(obj)  
                    if 'whatever' in obj[0]['word'].lower(): 
                      past.append(obj)
                    if 'however' in obj[0]['word'].lower(): 
                      past.append(obj)
                    if 'regardless' in obj[0]['word'].lower(): 
                      past.append(obj)
                    if 'notwithstanding' in obj[0]['word'].lower(): 
                      past.append(obj)  
                  if 'IN' in arr: 
                     arr.remove('IN')   
                     if 'in' in obj[0]['word'].lower(): 
                       past.append(obj) 
                     if 'if' in obj[0]['word'].lower():
                       past.append(obj)
                     if 'for' in obj[0]['word'].lower():
                       past.append(obj) 
                     if 'but' in obj[0]['word'].lower():
                       past.append(obj)
                     if 'because' in obj[0]['word'].lower():
                       past.append(obj)  
                     if 'although' in obj[0]['word'].lower():
                       past.append(obj)
               
              #arr.append(item) # uncomment for the full set
              #print(item)

              extract(item, arr)
  
        return arr

    results = extract(obj, arr)

    return modal, past, noun, verb

In [0]:
def separate_children(obj):
    arr = list()

    def separate(obj, arr):
        
        if isinstance(obj, dict):

            for k, v in obj.items():
              if k != 'children':
                if isinstance(v, (dict, list)):
                    separate(v, arr)
                elif k == 'word':      
                    arr.append(v)

        elif isinstance(obj, list):
            for item in obj:
                separate(item, arr)
         
        return arr

    results = separate(obj, arr)
    return results

In [0]:
def extract_constituents():
 for key, value in prediction.items():
  if (key=='hierplane_tree'):
    for key, value in value.items():
      if (key=='root'):
        for key, value in value.items():
          if (key == 'children'):
            return extract_modal(value)  

In [0]:
from nltk.tokenize import word_tokenize

def simplify(constit):
  gist = list();
  sentence = set()  # rebuild with checking for overlap
  for c in constit:
      #print(c)
      p = separate_children(c) # candidate phrases
      #print(p)
      for head in list(p):
        #print(head)
        phrase = word_tokenize(head)
        #print(phrase)
        words = set(phrase)
        if len(sentence.intersection(words)) < len(words):
          sentence.update(words)
          gist.extend(phrase)
  result = str()
  for word in gist:
    if word != '\'ve' and word != 'n\'t': # all exceptions
      result = result + ' '
    result = result + word  
  return result

In [0]:
import nltk
nltk.download('punkt')

original = "I don't think any of us---even economic gurus like Paul Krugman---really, truly understand just how bad it could've gotten 'on Main Street' if the stimulus bill had become hamstrung by a filibuster threat or recalcitrant conservadems, the way so much of our legislation has since."

modal = list()
nounm = list()
past = list() 
nounp = list()
nouns = list()

texts = original.rsplit(',')
for text in texts:
  text.replace('\"','\'')
  #print(text)

  prediction = predictor.predict(sentence=text)
  mod, pas, nou, ver = extract_constituents()

  for u in nou:
    nu = simplify(u) 
    if nu not in nouns:
      nouns.append(nu) 

  if len(mod)==0:
    for p in pas:
      pa = simplify(p)
      if pa not in past:
          past.append(pa) 

  for m in mod:
    md = simplify(m) # string gist

    for nm in reversed(sorted(nouns,key=len)):  
      if nm in md:   
        nounm.append(nm)
        nouns.remove(nm)
    
    for p in pas:
      pa = simplify(p)
     
      if pa in md and len(md) > len(pa):
        if md.index(pa) > 0 :
          md = md[0:md.index(pa)]
        else:
          md = md.replace(pa,'')    

      if md in pa and len(pa) > len(md):
        if pa.index(md) > 0 :
          pa = pa[0:pa.index(md)] 
        else:
          pa = pa.replace(md,'')  
            
      for np in reversed(sorted(nounp,key=len)):
        if np in pa and len(pa) > len(np): 
          if pa.index(np) > 0 :
            pa = pa[0:pa.index(np)]
          else:
            pa = pa.replace(np,'')  
          nounp.append(np)
          nouns.remove(np)    
      
      if pa not in past:
        past.append(pa) 

    if md not in modal:        
      modal.append(md) 

print(nounm)
print(nounp)

antecedents = past.copy()
consequents = modal.copy()

count=0 # max 10 times
while count < 10 and len(past) > 0 and len(modal) > 1: # consequent can't be empty!
  count += 1
  for m in sorted(modal,key=len):
    for p in reversed(sorted(past,key=len)): # start matching from smaller fragments
        if p in m and len(m) > len(p): 
          #print(m,p)
          if p in past and len(past) > 1:
            past.remove(p)  
          if m in modal and len(modal) > 1: 
            modal.remove(m)  

print(modal)
print(consequents)
print(antecedents)

for a in antecedents:
  for c in consequents:
    if c in a: 
      if a in past and len(past) > 1:
          past.remove(a)

for p1 in past:
  for p2 in past:
    if p2 in p1 and len(p1) > len(p2):
      past.remove(p2)   

for p2 in past:
  for p1 in past:
    if p1 in p2 and len(p2) > len(p1):
      past.remove(p1)                

print(past)

In [0]:
import nltk
nltk.download('punkt')

import pandas as pd

from tqdm import tqdm
from time import sleep

prefix = '/content/'
test_df = pd.read_csv(prefix + 'test.csv', header=None)
test_df = test_df.drop(index=0)
  
test_df = pd.DataFrame({
    'id':test_df[0],
    'text': test_df[1], 
    'labels': ''
})

count = test_df['id'].count()
print(count)


i = 0
j = 0
modal_df= pd.DataFrame(columns=['id','labels','text'])
past_df = pd.DataFrame(columns=['id','labels','text'])

for ind in tqdm(test_df.index): 
  original = test_df['text'][ind]

  modal = list()
  nounm = list()
  past = list() 
  nounp = list()
  nouns = list()

  texts = original.rsplit(',')
  for text in texts:
    text.replace('\"','\'')
    #print(text)

    prediction = predictor.predict(sentence=text)
    mod, pas, nou, ver = extract_constituents()

    for u in nou:
      nu = simplify(u) 
      if nu not in nouns:
        nouns.append(nu) 

    if len(mod)==0:
      for p in pas:
        pa = simplify(p)
        if pa not in past:
            past.append(pa) 

    for m in mod:
      md = simplify(m) # string gist

      for nm in reversed(sorted(nouns,key=len)):  
        if nm in md:   
          nounm.append(nm)
          nouns.remove(nm)
      
      for p in pas:
        pa = simplify(p)
      
        if pa in md and len(md) > len(pa):
          if md.index(pa) > 0 :
            md = md[0:md.index(pa)]
          else:
            md = md.replace(pa,'')    

        if md in pa and len(pa) > len(md):
          if pa.index(md) > 0 :
            pa = pa[0:pa.index(md)] 
          else:
            pa = pa.replace(md,'')  
              
        for np in reversed(sorted(nounp,key=len)):
          if np in pa and len(pa) > len(np): 
            if pa.index(np) > 0 :
              pa = pa[0:pa.index(np)]
            else:
              pa = pa.replace(np,'')  
            nounp.append(np)
            nouns.remove(np)    
        
        if pa not in past:
          past.append(pa) 

      if md not in modal:        
        modal.append(md) 

  antecedents = past.copy()
  consequents = modal.copy()

  count=0 # max 10 times
  while count < 10 and len(past) > 0 and len(modal) > 1: # consequent can't be empty!
    count += 1
    for m in sorted(modal,key=len):
      for p in reversed(sorted(past,key=len)): # start matching from smaller fragments
          if p in m and len(m) > len(p): 
            #print(m,p)
            if p in past and len(past) > 1:
              past.remove(p)  
            if m in modal and len(modal) > 1: 
              modal.remove(m)  

  for a in antecedents:
    for c in consequents:
      if c in a: 
        if a in past and len(past) > 1:
            past.remove(a)

  for p1 in past:
    for p2 in past:
      if p2 in p1 and len(p1) > len(p2):
        past.remove(p2)   

  for p2 in past:
    for p1 in past:
      if p1 in p2 and len(p2) > len(p1):
        past.remove(p1)  



  m_row = pd.Series(data={
    'id': test_df['id'][ind],
    'labels':'1', # gold label
    'text': test_df['text'][ind]
  }, name = str(i) )
  modal_df = modal_df.append(m_row)  
  i = i+1

  p_row = pd.Series(data={
    'id': test_df['id'][ind],
    'labels':'1', # gold label
    'text': test_df['text'][ind]
  }, name = str(j) )
  past_df = past_df.append(p_row)  
  j = j+1


  for m in modal:
    m_row = pd.Series(data={
      'id': test_df['id'][ind]+'A'+ str(i),
      'labels':'0',
      'text': m
    }, name = str(i) )
    modal_df = modal_df.append(m_row)  
    i = i+1
    
  for p in past:
    p_row = pd.Series(data={
      'id': test_df['id'][ind]+'C'+ str(j),
      'labels':'0',
      'text': p
    }, name = str(j) )
    past_df = past_df.append(p_row)  
    j = j+1

  sleep(1)  

past_df.to_csv(prefix+'antecedents.tsv', sep='\t', index=False, header=False)
modal_df.to_csv(prefix+'consequents.tsv', sep='\t', index=False, header=False)

  0%|          | 0/1950 [00:00<?, ?it/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
1950


  0%|          | 1/1950 [00:01<1:00:43,  1.87s/it]

In [0]:
!mv /content/antecedents.tsv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/
!mv /content/consequents.tsv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/